In [ ]:
import numpy as np

# Monkey patching: redefinindo np.float_ como np.float64
np.float_ = np.float64

#setup libs

%pip install yfinance

%pip install crewai
%pip install 'crewai[tools]'

%pip install langchain
%pip install langchain-openai

%pip install python-dotenv

%pip freeze > requirements.txt  

In [58]:
# import libs

import json
import os
from datetime import datetime

import yfinance as yf

from crewai import Agent, Task

from langchain.tools import Tool
from langchain_openai import ChatOpenAI

from dotenv import load_dotenv

In [59]:
# agent Yahoo Finance: Tool
def fetch_stock_price(ticket):
  stock = yf.download(ticket, start='2023-08-08', end='2024-08-20')
  return stock

yahooFinanceTool =Tool(
  name = 'Yahoo Finance Tool',
  description = 'Fetches stocks prices for {ticket} from the last year about a specific company stock from Yahoo Finance API',
  func = lambda ticket: fetch_stock_price(ticket)
)

In [60]:
# agent Yahoo Finance: LLM - GPT
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
llm = ChatOpenAI(model='gpt-3.5-turbo')

In [61]:
stockPriceAnalyst = Agent(
  role = 'Senior stock price Analyst',
  goal = 'Find the {ticket} stock price and analyses trends',
  backstory = '''You`re highly experienced in analyzing the price of an specific stock and make predictions about its future price.''',
  verbose = True,
  llm = llm,
  max_iter = 5,
  memory = True,
  tools = [yahooFinanceTool]
)

In [62]:
getStockPrice = Task(
  description = 'Analize the stock {ticket} price history and create a trend analyses of up, down or sideways',
  expected_output = '''Specify the current trend stock price - up, down or sideways.
  eg. stock= 'APPL, price UP'
  ''',
  agent = stockPriceAnalyst
)